# Henter data
Henter inn data om luftkvalitet fra Blindern i Oslo i løpet av hele 2017. 


In [157]:
import requests
import pandas as pd

# API URL
url = "https://api.nilu.no/aq/historical/2017-01-01/2017-12-31/59.89869/10.81495/3.0?components=no2;pm10"

#sender et get-kall til API-endepunketet og lagrer svaret response. 
response = requests.get(url)

def get_data():
    #Dersom statuskoden er 200 vil dataen lagres.
    if response.status_code == 200:
        
        data = response.json()  

        with open("luftkvalitet_2017.json","w") as f:
            json.dump(data, f, indent=4)
        print("Dataen er lagret i luftkvalitet_2017.json")
    #Dersom statuskoden er noe annet enn 200 vil vi få beskjed om at noe gikk galt. 
    else:
        print("noe gikk galt med API-kallet. Feilkoden er:", response.status_code)

get_data()






Dataen er lagret i luftkvalitet_2017.json


In [ ]:
import pandas as pd
from pandas import json_normalize
import json

# Åpne og les JSON-filen
with open('luftkvalitet_2017.json', 'r') as file:
    data = json.load(file)



def extract_component_data(data, component_name):
    # Filtrer dataene for den spesifikke komponenten (NO2 eller PM10)
    component_data = [item for item in data if item['component'] == component_name]
    
    # Normaliser 'values' for hver komponent
    df_list = [json_normalize(item['values'], sep='_') for item in component_data]
    
    # Slår sammen alle DataFrame-ene til én DataFrame
    df = pd.concat(df_list, ignore_index=True)
    
    # Legg til en kolonne for komponenten
    df['component'] = component_name
    
    return df


df_no2 = extract_component_data(data, 'NO2')

df_no2 = df_no2.rename(columns={'value': 'NO2'})

df_pm10 = extract_component_data(data, 'PM10')

df_pm10 = df_pm10.rename(columns={'value': 'PM10'})

df_no2['PM10'] = df_pm10['PM10']








maalinger_2017=df_no2




maalinger_2017







,fromTime,toTime,NO2,qualityControlled,index,color,component,PM10
0,2017-01-01T00:00:00+01:00,2017-01-01T01:00:00+01:00,33.656246,True,1,6ee86e,NO2,305.36
1,2017-01-01T01:00:00+01:00,2017-01-01T02:00:00+01:00,66.923648,True,1,6ee86e,NO2,258.06
2,2017-01-01T02:00:00+01:00,2017-01-01T03:00:00+01:00,45.079216,True,1,6ee86e,NO2,103.84
3,2017-01-01T03:00:00+01:00,2017-01-01T04:00:00+01:00,63.750189,True,1,6ee86e,NO2,54.23
4,2017-01-01T04:00:00+01:00,2017-01-01T05:00:00+01:00,60.040693,True,1,6ee86e,NO2,35.75
...,...,...,...,...,...,...,...,...
8689,2017-12-30T19:00:00+01:00,2017-12-30T20:00:00+01:00,51.413664,True,1,6ee86e,NO2,12.43
8690,2017-12-30T20:00:00+01:00,2017-12-30T21:00:00+01:00,27.269724,True,1,6ee86e,NO2,14.30
8691,2017-12-30T21:00:00+01:00,2017-12-30T22:00:00+01:00,20.334475,True,1,6ee86e,NO2,12.65
8692,2017-12-30T22:00:00+01:00,2017-12-30T23:00:00+01:00,23.900351,True,1,6ee86e,NO2,11.55


# Fjerner unødvendige kolonner
Fjerner kolonnene qualityControlled, index, color, component for å gjøre tabellen mer oversiktelig. Legger også til kolonnen "unit" som oppgir enheten til PM10- og NO2-målingene. I tilegg runder jeg av måleverdiene til maks 3 desimaler. 

In [159]:
# Fjerner unødvendige kolonner
maalinger_2017= maalinger_2017.drop(['qualityControlled','index','color','component'], axis=1)
#legger til kolonnen unit
maalinger_2017['unit']='µg/m³'
#endrer til maks tre desimaler
maalinger_2017[['NO2', 'PM10']] = maalinger_2017[['NO2', 'PM10']].round(3)



maalinger_2017








,fromTime,toTime,NO2,PM10,unit
0,2017-01-01T00:00:00+01:00,2017-01-01T01:00:00+01:00,33.656,305.36,µg/m³
1,2017-01-01T01:00:00+01:00,2017-01-01T02:00:00+01:00,66.924,258.06,µg/m³
2,2017-01-01T02:00:00+01:00,2017-01-01T03:00:00+01:00,45.079,103.84,µg/m³
3,2017-01-01T03:00:00+01:00,2017-01-01T04:00:00+01:00,63.750,54.23,µg/m³
4,2017-01-01T04:00:00+01:00,2017-01-01T05:00:00+01:00,60.041,35.75,µg/m³
...,...,...,...,...,...
8689,2017-12-30T19:00:00+01:00,2017-12-30T20:00:00+01:00,51.414,12.43,µg/m³
8690,2017-12-30T20:00:00+01:00,2017-12-30T21:00:00+01:00,27.270,14.30,µg/m³
8691,2017-12-30T21:00:00+01:00,2017-12-30T22:00:00+01:00,20.334,12.65,µg/m³
8692,2017-12-30T22:00:00+01:00,2017-12-30T23:00:00+01:00,23.900,11.55,µg/m³


# Endrer på tidssoppsettet
 Endrer på tidsoppsettet ettersom tidsoppsetett med 'fromTime' og 'toTime' ikker var oversiktelig. Konverterer verdiene i 'fromTime' kolonnen fra tekststrenger til datetime-objekter slik at det blir lettere å jobbe med.  Lager kolonnene 'Date' som oppgir dato og 'time' som oppgir klokkeslett for målingene. Etter at de nye kolonnene er opprettet sletter vi kolonnene 'fromTime' og 'toTime'. Vi velger så å bytte plass på kolonne slik at vi får de i rekkefølgen 'Date', 'Time', 'NO2', 'PM10','unit'. 
 

In [160]:
#Konverterer verdiene i 'fromTime' kolonnen fra tekststrenger til datetime-objekter.
maalinger_2017['fromTime'] = pd.to_datetime(maalinger_2017['fromTime'])

#Oppretter en ny kolonne 'Date' som viser datoen til målingene. 
maalinger_2017['Date'] = maalinger_2017['fromTime'].dt.date

#Oppretter en ny kolonne 'Time' som viser klokkeslettet for målingene. 
maalinger_2017['Time'] = maalinger_2017['fromTime'].dt.strftime('%H:%M')

#Sletter kolonnene 'fromTime' og 'toTime'.
maalinger_2017= maalinger_2017.drop(['fromTime','toTime'], axis=1)

#Endrer rekkefølge på kolonnene
maalinger_2017 = maalinger_2017[['Date', 'Time', 'NO2', 'PM10','unit']]


maalinger_2017.to_csv('../data/maalinger_2017.csv')






In [161]:
maalinger_2017

,Date,Time,NO2,PM10,unit
0,2017-01-01,00:00,33.656,305.36,µg/m³
1,2017-01-01,01:00,66.924,258.06,µg/m³
2,2017-01-01,02:00,45.079,103.84,µg/m³
3,2017-01-01,03:00,63.750,54.23,µg/m³
4,2017-01-01,04:00,60.041,35.75,µg/m³
...,...,...,...,...,...
8689,2017-12-30,19:00,51.414,12.43,µg/m³
8690,2017-12-30,20:00,27.270,14.30,µg/m³
8691,2017-12-30,21:00,20.334,12.65,µg/m³
8692,2017-12-30,22:00,23.900,11.55,µg/m³


Sjekker for manglende verdier, og finner ut at vi ikke mangler noe. I tilegg vil jeg sjekke om målingene av NO2 og PM10 gir mening. Jeg sjekker om alle målingene for hver time er under 400 µg/m. Verdier som er høyere enn 400 µg/m blir ikke lenger kategorisert av miljødirektoratets miljøklasser etterom de regnes som urealistiske. Men finner ut at alle målingene er innenfor gyldighetsområdet. 

In [162]:
#sjekker for missing values
missing_values = maalinger_2017.isnull().sum()

# Definer grensen for unormale verdier
unormalt_no2 = maalinger_2017[maalinger_2017['NO2'] > 400]  # Ekstreme verdier for NO2
unormalt_pm10 = maalinger_2017[maalinger_2017['PM10'] > 400]  # Ekstreme verdier for PM10

print(f"Ekstreme NO2 verdier:\n{unormalt_no2}")
print(f"Ekstreme PM10 verdier:\n{unormalt_pm10}")

print(missing_values)


maalinger_2017





Ekstreme NO2 verdier:
Empty DataFrame
Columns: [Date, Time, NO2, PM10, unit]
Index: []
Ekstreme PM10 verdier:
Empty DataFrame
Columns: [Date, Time, NO2, PM10, unit]
Index: []
Date    0
Time    0
NO2     0
PM10    0
unit    0
dtype: int64


,Date,Time,NO2,PM10,unit
0,2017-01-01,00:00,33.656,305.36,µg/m³
1,2017-01-01,01:00,66.924,258.06,µg/m³
2,2017-01-01,02:00,45.079,103.84,µg/m³
3,2017-01-01,03:00,63.750,54.23,µg/m³
4,2017-01-01,04:00,60.041,35.75,µg/m³
...,...,...,...,...,...
8689,2017-12-30,19:00,51.414,12.43,µg/m³
8690,2017-12-30,20:00,27.270,14.30,µg/m³
8691,2017-12-30,21:00,20.334,12.65,µg/m³
8692,2017-12-30,22:00,23.900,11.55,µg/m³


I dataen vi har hentet inn har vi målinger hver time. Jeg vil nå lage en tabell med gjennomsnittet hvor hver dag slik at det blir mer oversiktelig. 

# Gjennomsnitt per dag
Ettersom dataen som er hentet inneholder målinger for hver time, lager vi en ny oversikt med gjennomsnittet av målingene per døgn slik at den blir mer oversiktelig. 


In [165]:
# Grupper etter 'Date' og beregner gjennomsnittet for hver dag
daglig_gjennomsnitt = maalinger_2017.groupby('Date')[['NO2', 'PM10']].mean().reset_index()

daglig_gjennomsnitt['Enhet'] = 'µg/m³'


# Runder av verdiene til 3 desimaler
daglig_gjennomsnitt[['NO2', 'PM10']] = daglig_gjennomsnitt[['NO2', 'PM10']].round(3)

daglig_gjennomsnitt.to_csv('../data/daglig_gjennomsnitt_2017.csv')



